# Exact Diagonalization of $J$-$K$ Hamiltonian
    Alan Morningstar
    May 2017

## Include Source

In [1]:
include("utils.jl");
include("lattice.jl");
include("basis.jl");
include("sparseHermitian.jl");
include("sparseHam.jl");
include("sparseS2.jl");

## Main

#### specify parameters

In [2]:
# square lattice length
const Lx = 6;
const Ly = 4;
# number of sites
const N = Lx*Ly;
# NN coupling
const J1 = 1.0;
# NNN coupling
const J2 = 0.0;
# plaquette coupling
const K = 0.0;

# choose Sz sector by specifying number of 1s in basis states
const n1 = convert(Int64,N/2);
# choose kx,ky by specifying mi such that mi is in 0:Li-1
const mx = 0;
const my = 0;
const kx = 2*pi*mx/Lx;
const ky = 2*pi*my/Ly;

# number of eigenvalues desired
const numEigs = 6;
# a tolerance for error
const tolerance = 10.^(-5.);
# ritzVec = true if you want the eigenvectors returned too
const ritzVec = true;
# number of Krylov vectors in eigenvalue calculation
const numKrylovVecs = 20;
# maximum number of iterations to converge eigenvalues
const maxIter = 300;

# plaquette (x,y) vectors, locating p1,p2,p3,p4 on the plaquette of the p1 site and p1D,p2D,p1L,p3L on adjacent plaquettes
const neighborVectors = [(0,0),(1,0),(0,1),(1,1),(0,-1),(1,-1),(-1,0),(-1,1)];

#### define lattice, symmetry sector, and basis

In [3]:
# define the lattice
const l = lattice(Lx,Ly,neighborVectors);

In [4]:
# specify symmetry sector
const s = sector(n1,kx,ky);

In [6]:
# construct the basis
# 0.34 seconds for 6x4 lattice, basis is ~0.002 GB
@time const basis = SzkxkyBasis(l,s);
println("Dimension of reduced Hilbert space is ",basis.dim,".");

  0.373373 seconds (2.70 M allocations: 870.507 MiB, 9.80% gc time)
Dimension of reduced Hilbert space is 112800.


In [7]:
# using ProfileView;

In [8]:
# Profile.clear();
# @profile SzkxkyBasis(l,s);
# ProfileView.view()

#### build the Hamiltonian

In [10]:
# couplings type to make passing J1,K easier
const c = couplings(J1,J2,K);

# build the sparse Hamiltonian
# takes 1.02 seconds for 6x4 Heisenberg model, H contains ~0.03 GB
@time const H = constructSparseHam(basis,c,s,l);

  1.022912 seconds (5.41 M allocations: 420.519 MiB, 1.61% gc time)


In [19]:
# using ProfileView;

In [20]:
# Profile.clear();
# @profile constructSparseHam(basis,c,s,l);
# ProfileView.view()

#### find eigenvalues and eigenvectors

In [13]:
# compute eigenvalues
# ~3.95 seconds for 6x4, numEigs=6, which=:SR, Heisenberg Hamiltonian
@time eigsResult = eigs(H; nev=numEigs,ncv=numKrylovVecs,maxiter=maxIter, which=:SR, tol=tolerance, ritzvec=ritzVec);

  3.952152 seconds (870 allocations: 51.684 MiB, 0.22% gc time)


In [14]:
# print energies
println("Energies are: ",real(eigsResult[1]),".");
# print algorithm performance
println("Number of iterations = ",eigsResult[3]);
println("Number of matrix-vector multiplications = ",eigsResult[4]);

Energies are: [-16.5525, -15.3109, -14.1597, -13.7363, -13.0826, -12.8661].
Number of iterations = 6
Number of matrix-vector multiplications = 9


#### build the $S^2$ operator

In [16]:
# takes 3.60 seconds for 6x4 lattice, S2 contains ~0.17 GB
@time const S2 = constructSparseS2(basis,s,l);

  3.603707 seconds (55 allocations: 162.437 MiB, 1.96% gc time)


#### save spectral data
Save in format |  E  |  S(S+1)  |  Sz  |  mx  |  my  |

In [18]:
using DataFrames;

In [19]:
# energies
EData = real(eigsResult[1]);
# Sz values
SzData = fill((N-2*n1)/2,numEigs);
# mx values
mxData = fill(mx,numEigs);
# my values
myData = fill(my,numEigs);

In [21]:
# S(S+1) values
S2Data = zeros(Int64,numEigs);
psi = Array{Complex128,1}(basis.dim);
S2psi = Array{Complex128,1}(basis.dim);
for i in 1:numEigs
    psi = eigsResult[2][:,i];
    A_mul_B!(S2psi,S2,psi);
    S2Data[i] = round(Int64,real(dot(psi,S2psi))[1]);
end;

In [22]:
# create DataFrame
df = DataFrame(E=EData,Ssqrd=S2Data,Sz=SzData,mx=mxData,my=myData);

In [23]:
println(df)

6×5 DataFrames.DataFrame
│ Row │ E        │ Ssqrd │ Sz  │ mx │ my │
├─────┼──────────┼───────┼─────┼────┼────┤
│ 1   │ -16.5525 │ 0     │ 0.0 │ 0  │ 0  │
│ 2   │ -15.3109 │ 6     │ 0.0 │ 0  │ 0  │
│ 3   │ -14.1597 │ 0     │ 0.0 │ 0  │ 0  │
│ 4   │ -13.7363 │ 2     │ 0.0 │ 0  │ 0  │
│ 5   │ -13.0826 │ 0     │ 0.0 │ 0  │ 0  │
│ 6   │ -12.8661 │ 6     │ 0.0 │ 0  │ 0  │


In [24]:
# dataFileName = "specData/Sz=" * string(SzData[1]) * "_mx=" * string(mxData[1]) * "_my=" * string(myData[1]) * ".csv";

In [25]:
# writetable(dataFileName, df)
# df1 = readtable("specData/Sz=0.0_mx=0_my=0.csv");